In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize

import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
from gensim.models import Word2Vec
import gensim 

from  nltk import FreqDist

In [499]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
#nltk.download('genesis')
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
genesis_ic = wn.ic(genesis, False, 0.0)
import re
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score
from random import randint

In [3]:
#read in one dataset
df=pd.read_csv('epicurious')

In [4]:
df=df.dropna()

In [60]:
df.ingredients.head(10)

0    ['1 1/2 cups whipping cream', '2 medium onions...
1    ['2 1/2 cups (lightly packed) fresh basil leav...
2    ['1/4 cup soy sauce', '1 tablespoon sugar', '2...
3    ['6 long parsley sprigs, divided', '1 3/4 cups...
4    ['4 teaspoons olive oil', '1/2 cup finely chop...
5    ['Nonstick vegetable oil spray', '3 cups all-p...
6    ['4 6- to 7-ounce beef tenderloin steaks (each...
7    ['1 large ripe peach', '2 tablespoons sugar', ...
8    ['2 cups fresh tomato and/or carrot juice (fro...
9    ['1 cup water', '2/3 cup buttermilk', '1/3 cup...
Name: ingredients, dtype: object

In [6]:
df=df.reset_index(drop=True)

In [7]:
df.head()

,Unnamed: 0,calories,categories,date,desc,directions,fat,ingredients,protein,rating,sodium,title
0,1,403.0,"['Food Processor', 'Onion', 'Pork', 'Bake', 'B...",2004-08-20 04:00:00.000,This uses the same ingredients found in boudin...,['Combine first 9 ingredients in heavy medium ...,23.0,"['1 1/2 cups whipping cream', '2 medium onions...",18.0,4.375,1439.0,Boudin Blanc Terrine with Red Onion Confit
1,5,948.0,"['Sandwich', 'Food Processor', 'Tomato', 'Kid-...",2004-08-20 04:00:00.000,This recipe can be prepared in 45 minutes or l...,"['Mix basil, mayonnaise and butter in processo...",79.0,['2 1/2 cups (lightly packed) fresh basil leav...,19.0,4.375,1042.0,The Best Blts
2,8,170.0,"['Beef', 'Ginger', 'Sauté', 'Stir-Fry', 'Quick...",2004-08-20 04:00:00.000,Bulgogi,"['Stir together soy sauce, sugar, sesame oil, ...",10.0,"['1/4 cup soy sauce', '1 tablespoon sugar', '2...",7.0,4.375,1272.0,Korean Marinated Beef
3,9,602.0,"['Salad', 'Mustard', 'Potato', 'Picnic', 'Lunc...",2008-10-23 22:24:26.000,Transform your picnic into un pique-nique to r...,['Chop enough parsley leaves to measure 1 tabl...,41.0,"['6 long parsley sprigs, divided', '1 3/4 cups...",23.0,3.750,1696.0,Ham Persillade with Mustard Potato Salad and M...
4,10,256.0,"['Milk/Cream', 'Dairy', 'Side', 'Thanksgiving'...",2004-08-20 04:00:00.000,Simmering the yams fills them with flavor and ...,['Heat oil in heavy large skillet over medium-...,5.0,"['4 teaspoons olive oil', '1/2 cup finely chop...",4.0,3.750,30.0,"Yams Braised with Cream, Rosemary and Nutmeg"


In [8]:
# get stop words
from nltk.corpus import stopwords

stopwords_list = stopwords.words('english')

In [9]:
#use string library
import string

In [10]:
#add punctuation to stopwords list
stopwords_list += list(string.punctuation)

In [11]:
#add words that aren't in the NLTK stopwords list
stopwords_list += ['cup','kosher','lb','oz','diced','sliced','pitted','peeled','seasoned','teaspoon','teaspoons','tablespoons', 'tablespoon', 'pound','cups','half','salt','pepper','chopped','bowl','directions','preheat','pieces','sliced','fresh','freshly','bunch','cut']
stopwords_list+=['1/2-inch-thick','1/4-inch-thick','s','12','1','2','3','4','5','6','7','8','9','1/2','1/3','1/4','2/3','3/4','1/8','small','large','medium']

In [12]:
def process_article(article):
    tokens = word_tokenize(article)
    stopwords_removed = [token.lower() for token in tokens if token not in stopwords_list]
    return stopwords_removed 

In [13]:
#df['a'] = df['a'].apply(lambda x: x + 1)
df['Ingredient_join']=df['ingredients'].apply(lambda x:' '.join(x))
 
# df['ingredients'].astype(str).str.cat(df['ingredients'],sep=' ')

In [14]:
def process_sent(article):
    tokens = sent_tokenize(article)
    stopwords_removed = [token.lower() for token in tokens if token not in stopwords_list]
    return stopwords_removed 

In [15]:
#vectorize_list=df['Ingredient_join'].apply(lambda x: process_article(x))

In [16]:
df['processed_titles']=df['title'].apply(lambda x: process_article(x))

In [17]:
df['title_join']=df['processed_titles'].apply(lambda x:' '.join(x))

In [18]:
vectorize_title=df['title_join'].apply(lambda x: process_article(x))

In [19]:
# this is the best format so far
df['processed_ingredients']=df['ingredients'].apply(lambda x: process_article(x))

In [20]:
df['ingredients_join']=df['processed_ingredients'].apply(lambda x:' '.join(x))

In [21]:
df['processed_directions']=df['directions'].apply(lambda x: process_article(x))

In [22]:
df['directions_join']=df['processed_directions'].apply(lambda x:' '.join(x))

In [23]:
# i think this is the tokenized directions
df['processed_directions'].head()

0    ['combine, first, ingredients, heavy, saucepan...
1    ['mix, basil, mayonnaise, butter, processor, b...
2    ['stir, together, soy, sauce, sugar, sesame, o...
3    ['chop, enough, parsley, leaves, measure, rese...
4    ['heat, oil, heavy, skillet, medium-high, heat...
Name: processed_directions, dtype: object

In [24]:
df['recipe_combo']=df['processed_titles']+df['processed_ingredients']+df['processed_directions']

In [ ]:
#df['vector_ingred_lines']=df['processed_ingredients'].apply(lambda x: model_5[x])

In [26]:
df.head()

,Unnamed: 0,calories,categories,date,desc,directions,fat,ingredients,protein,rating,sodium,title,Ingredient_join,processed_titles,title_join,processed_ingredients,ingredients_join,processed_directions,directions_join,recipe_combo
0,1,403.0,"['Food Processor', 'Onion', 'Pork', 'Bake', 'B...",2004-08-20 04:00:00.000,This uses the same ingredients found in boudin...,['Combine first 9 ingredients in heavy medium ...,23.0,"['1 1/2 cups whipping cream', '2 medium onions...",18.0,4.375,1439.0,Boudin Blanc Terrine with Red Onion Confit,[ ' 1 1 / 2 c u p s w h i p p i n g c ...,"[boudin, blanc, terrine, red, onion, confit]",boudin blanc terrine red onion confit,"[whipping, cream, onions, bay, leaves, whole, ...",whipping cream onions bay leaves whole cloves ...,"['combine, first, ingredients, heavy, saucepan...",'combine first ingredients heavy saucepan add ...,"[boudin, blanc, terrine, red, onion, confit, w..."
1,5,948.0,"['Sandwich', 'Food Processor', 'Tomato', 'Kid-...",2004-08-20 04:00:00.000,This recipe can be prepared in 45 minutes or l...,"['Mix basil, mayonnaise and butter in processo...",79.0,['2 1/2 cups (lightly packed) fresh basil leav...,19.0,4.375,1042.0,The Best Blts,[ ' 2 1 / 2 c u p s ( l i g h t l y p ...,"[the, best, blts]",the best blts,"[lightly, packed, basil, leaves, mayonnaise, b...",lightly packed basil leaves mayonnaise butter ...,"['mix, basil, mayonnaise, butter, processor, b...",'mix basil mayonnaise butter processor basil f...,"[the, best, blts, lightly, packed, basil, leav..."
2,8,170.0,"['Beef', 'Ginger', 'Sauté', 'Stir-Fry', 'Quick...",2004-08-20 04:00:00.000,Bulgogi,"['Stir together soy sauce, sugar, sesame oil, ...",10.0,"['1/4 cup soy sauce', '1 tablespoon sugar', '2...",7.0,4.375,1272.0,Korean Marinated Beef,"[ ' 1 / 4 c u p s o y s a u c e ' , ' ...","[korean, marinated, beef]",korean marinated beef,"[soy, sauce, sugar, asian, sesame, oil, scalli...",soy sauce sugar asian sesame oil scallions whi...,"['stir, together, soy, sauce, sugar, sesame, o...",'stir together soy sauce sugar sesame oil whit...,"[korean, marinated, beef, soy, sauce, sugar, a..."
3,9,602.0,"['Salad', 'Mustard', 'Potato', 'Picnic', 'Lunc...",2008-10-23 22:24:26.000,Transform your picnic into un pique-nique to r...,['Chop enough parsley leaves to measure 1 tabl...,41.0,"['6 long parsley sprigs, divided', '1 3/4 cups...",23.0,3.750,1696.0,Ham Persillade with Mustard Potato Salad and M...,[ ' 6 l o n g p a r s l e y s p r i g s ...,"[ham, persillade, mustard, potato, salad, mash...",ham persillade mustard potato salad mashed peas,"[long, parsley, sprigs, divided, reduced-sodiu...",long parsley sprigs divided reduced-sodium chi...,"['chop, enough, parsley, leaves, measure, rese...",'chop enough parsley leaves measure reserve ch...,"[ham, persillade, mustard, potato, salad, mash..."
4,10,256.0,"['Milk/Cream', 'Dairy', 'Side', 'Thanksgiving'...",2004-08-20 04:00:00.000,Simmering the yams fills them with flavor and ...,['Heat oil in heavy large skillet over medium-...,5.0,"['4 teaspoons olive oil', '1/2 cup finely chop...",4.0,3.750,30.0,"Yams Braised with Cream, Rosemary and Nutmeg",[ ' 4 t e a s p o o n s o l i v e o i l ...,"[yams, braised, cream, rosemary, nutmeg]",yams braised cream rosemary nutmeg,"[olive, oil, finely, shallots, minced, rosemar...",olive oil finely shallots minced rosemary poun...,"['heat, oil, heavy, skillet, medium-high, heat...",'heat oil heavy skillet medium-high heat add s...,"[yams, braised, cream, rosemary, nutmeg, olive..."


In [27]:
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec

model_combo = Word2Vec(df['recipe_combo'], min_count=20)

In [530]:
test_run=df[df['ingredients'].str.contains('cow')]

In [531]:
test_run=test_run.reset_index(drop=True)

In [532]:
test_run

,Unnamed: 0,calories,categories,date,desc,directions,fat,ingredients,protein,rating,sodium,title,Ingredient_join,processed_titles,title_join,processed_ingredients,ingredients_join,processed_directions,directions_join,recipe_combo
0,1004,591.0,"['Milk/Cream', 'Chicken', 'Ginger', 'Onion', '...",2006-05-16 20:12:55.000,Ca ri ga,['1. Combine 2 tablespoons of the curry powder...,21.0,"['3 tablespoons curry powder, preferably Three...",54.0,5.000,1735.0,Chicken Curry with Sweet Potatoes,[ ' 3 t a b l e s p o o n s c u r r y p ...,"[chicken, curry, sweet, potatoes]",chicken curry sweet potatoes,"[curry, powder, preferably, three, golden, bel...",curry powder preferably three golden bells bra...,"[combine, curry, powder, add, chicken, turn, c...",combine curry powder add chicken turn coat mea...,"[chicken, curry, sweet, potatoes, curry, powde..."
1,14116,1453.0,"['Soup/Stew', 'Cheese', 'Chicken', 'Stew', 'Ba...",2006-05-16 20:12:06.000,"A slightly firm, mellow triple-crème, Mt. Tam ...",['Soak chickpeas in water to cover by 2 inches...,91.0,"['1 pound dried chickpeas (2 1/2 cups), picked...",79.0,4.375,1992.0,"Chickpea, Chorizo, and Chicken Stew with Mt. T...",[ ' 1 p o u n d d r i e d c h i c k p e ...,"[chickpea, chorizo, chicken, stew, mt, tam, ch...",chickpea chorizo chicken stew mt tam cheese,"[dried, chickpeas, picked, tomatoes, black, pe...",dried chickpeas picked tomatoes black pepperco...,"['soak, chickpeas, water, cover, inches, hours...",'soak chickpeas water cover inches hours drain...,"[chickpea, chorizo, chicken, stew, mt, tam, ch..."
2,16735,882.0,"['Garlic', 'Onion', 'Kid-Friendly', 'Dinner', ...",2015-03-26 04:00:00.000,Mullen serves these robustly flavored meatball...,"['Toast chiles, red pepper flakes, and cumin s...",80.0,"['3 pasilla chiles, seeds removed, chopped', '...",29.0,4.375,1558.0,Chile-Cumin Lamb Meatballs with Yogurt and Cuc...,"[ ' 3 p a s i l l a c h i l e s , s e e ...","[chile-cumin, lamb, meatballs, yogurt, cucumber]",chile-cumin lamb meatballs yogurt cucumber,"[pasilla, chiles, seeds, removed, crushed, red...",pasilla chiles seeds removed crushed red flake...,"['toast, chiles, red, flakes, cumin, seeds, dr...",'toast chiles red flakes cumin seeds dry skill...,"[chile-cumin, lamb, meatballs, yogurt, cucumbe..."
3,19025,310.0,"['Cookies', 'Dessert', 'Butter', 'Bon Appétit']",2014-08-19 04:00:00.000,The caramelized milk jam makes for a decadent ...,"['Using an electric mixer on high speed, beat ...",18.0,"['1 1/2 cups (3 sticks) unsalted butter, room ...",4.0,0.000,295.0,Nana's Butter Cookies with Milk-Jam Filling,[ ' 1 1 / 2 c u p s ( 3 s t i c k s ) ...,"[nana, 's, butter, cookies, milk-jam, filling]",nana 's butter cookies milk-jam filling,"[sticks, unsalted, butter, room, temperature, ...",sticks unsalted butter room temperature powder...,"['using, electric, mixer, high, speed, beat, b...",'using electric mixer high speed beat butter p...,"[nana, 's, butter, cookies, milk-jam, filling,..."


In [533]:

item=randint(0,len(test_run))

In [534]:
item

2

In [535]:
test_ingredients=test_run['processed_ingredients'][item]

In [536]:
test_ingredients

['pasilla',
 'chiles',
 'seeds',
 'removed',
 'crushed',
 'red',
 'flakes',
 'cumin',
 'seeds',
 'sherry',
 'vinegar',
 'red',
 'wine',
 'vinegar',
 'sweet',
 'smoked',
 'paprika',
 'garlic',
 'clove',
 'olive',
 'oil',
 "'kosher",
 'onion',
 'garlic',
 'cloves',
 'cloves',
 'clove',
 'finely',
 'grated',
 'parsley',
 'oregano',
 'sage',
 'fennel',
 'seeds',
 'ground',
 'coriander',
 'ground',
 'cumin',
 'egg',
 'pounds',
 'ground',
 'lamb',
 'rice',
 'flour',
 'all-purpose',
 'flour',
 '4–5',
 'olive',
 'oil',
 'divided',
 'english',
 'hothouse',
 'cucumber',
 'thinly',
 'finely',
 'grated',
 'lemon',
 'zest',
 'lemon',
 'juice',
 'sherry',
 'vinegar',
 'red',
 'wine',
 'vinegar',
 "'freshly",
 'ground',
 'black',
 '``',
 'plain',
 "sheep's-milk",
 'cow',
 "'s",
 'whole-milk',
 'yogurt',
 "''",
 'mint',
 "'crushed",
 'red',
 'flakes',
 'serving',
 'a',
 'spice',
 'mill',
 'mortar',
 'pestle']

In [556]:
key=test_ingredients[0]

In [570]:
pwd


'/Users/denisdunn'

In [557]:
key

'pasilla'

In [558]:
test_run.iloc[item]

Unnamed: 0                                                           16735
calories                                                               882
categories               ['Garlic', 'Onion', 'Kid-Friendly', 'Dinner', ...
date                                               2015-03-26 04:00:00.000
desc                     Mullen serves these robustly flavored meatball...
directions               ['Toast chiles, red pepper flakes, and cumin s...
fat                                                                     80
ingredients              ['3 pasilla chiles, seeds removed, chopped', '...
protein                                                                 29
rating                                                               4.375
sodium                                                                1558
title                    Chile-Cumin Lamb Meatballs with Yogurt and Cuc...
Ingredient_join          [ ' 3   p a s i l l a   c h i l e s ,   s e e ...
processed_titles         

In [559]:
answer=model_combo.most_similar(positive=[test_ingredients[0]])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [560]:
final=answer[0][0]

In [561]:
key

'pasilla'

In [562]:
similar_words=model_combo.most_similar(positive=[key])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [563]:
similar_words

[('árbol', 0.9101869463920593),
 ('guajillo', 0.9064054489135742),
 ('ancho', 0.8787860870361328),
 ('chiles*', 0.8413679599761963),
 ('habanero', 0.809214174747467),
 ('anaheim', 0.7972064018249512),
 ('bonnet', 0.7896746397018433),
 ('chilies*', 0.7855426073074341),
 ('poblano', 0.7792425751686096),
 ('fresno', 0.7619127631187439)]

In [564]:
similar_list=[]
words=[]
for word in range(1,len(similar_words)):
    similar_list.append(model_combo.similarity(similar_words[word][0], final))
    words.append(similar_words[word][0])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  after removing the cwd from sys.path.


In [565]:
layered_relationships=dict(zip(words,similar_list))

In [566]:
layered_relationships

{'guajillo': 0.8780029,
 'ancho': 0.8374705,
 'chiles*': 0.8040782,
 'habanero': 0.77476287,
 'anaheim': 0.7509365,
 'bonnet': 0.8057234,
 'chilies*': 0.7001704,
 'poblano': 0.69749457,
 'fresno': 0.78252935}

In [567]:
final_2 = max(layered_relationships.keys(), key=(lambda k: layered_relationships[k]))

In [568]:
final_2

'guajillo'

In [569]:
test_run.iloc[item]['title'],test_run.iloc[item]['ingredients']

('Chile-Cumin Lamb Meatballs with Yogurt and Cucumber ',
 '[\'3 pasilla chiles, seeds removed, chopped\', \'1 1/2 teaspoons crushed red pepper flakes\', \'1 tablespoon cumin seeds\', \'1/4 cup Sherry vinegar or red wine vinegar\', \'1 tablespoon sweet smoked paprika\', \'1 garlic clove, chopped\', \'1/2 cup olive oil\', \'Kosher salt\', \'1/4 small onion, chopped\', \'8 garlic cloves, 7 cloves chopped, 1 clove finely grated\', \'1 tablespoon chopped fresh parsley\', \'1 teaspoon chopped fresh oregano\', \'1 teaspoon chopped fresh sage\', \'3/4 teaspoon fennel seeds\', \'3/4 teaspoon ground coriander\', \'3/4 teaspoon ground cumin\', \'1 tablespoon kosher salt\', \'1 large egg\', \'1 1/4 pounds ground lamb\', \'1 tablespoon rice flour or all-purpose flour\', \'4–5 tablespoons olive oil, divided\', \'1/2 English hothouse cucumber, thinly sliced\', \'1/2 teaspoon finely grated lemon zest\', \'2 tablespoons fresh lemon juice\', \'1 1/2 tablespoons Sherry vinegar or red wine vinegar\', \'Fr

In [551]:
key

'red'

In [552]:
test_run.iloc[item]['title'], test_run.iloc[item]['directions'].replace(key,final_2), test_run.iloc[item]['ingredients'].replace(key,final_2)

('Chile-Cumin Lamb Meatballs with Yogurt and Cucumber ',
 '[\'Toast chiles, bonito pepper flakes, and cumin seeds in a dry small skillet over medium heat, tossing occasionally, until fragrant and cumin seeds are golden, about 3 minutes.\', \'Let cool. Working in batches, finely grind in spice mill, then transfer to a blender. Add vinegar, paprika, and garlic to blender and blend until smooth. With motor running, gradually stream in oil and blend until combined. Transfer to a large bowl; season with salt.\', \'Do ahead: Chile sauce can be made 2 days ahead. Cover and chill.\', \'Pulse onion, chopped garlic, parsley, oregano, sage, fennel seeds, coriander, cumin, and 1 tablespoon salt in a food processor, scraping down sides as needed, until finely chopped. Add egg, lamb, and flour and pulse until evenly combined. Form lamb mixture into 1 1/2" balls.\', \'Heat 2 tablespoons oil in a large skillet, preferably cast iron, over medium-high. Working in 2 batches and adding another 1 tablespoo

In [359]:
model_combo.similarity(['yellow'], 'red-pepper')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


array([0.26060563], dtype=float32)

In [360]:
for change in test_run['recipe_combo'][29]:
    change.replace(key,final)

In [361]:
test_run['recipe_combo'][29]

['chinese-style',
 'spinach',
 'mushroom',
 'soup',
 '14',
 '1/2-ounce',
 'cans',
 'vegetable',
 'broth',
 'water',
 'soy',
 'sauce',
 'rice',
 'vinegar',
 'apple',
 'cider',
 'vinegar',
 'oriental',
 'sesame',
 'oil',
 'minced',
 'garlic',
 'minced',
 'ginger',
 'thinly',
 'mushrooms',
 'ounces',
 'firm',
 'tofu',
 '1/2-inch',
 '10-ounce',
 'bag',
 'ready-to-use',
 'spinach',
 'leaves',
 'green',
 'onions',
 "'combine",
 'first',
 'ingredients',
 'saucepan',
 'bring',
 'boil',
 'medium-high',
 'heat',
 'add',
 'mushrooms',
 'simmer',
 'mushrooms',
 'tender',
 'minutes',
 'add',
 'tofu',
 'spinach',
 'green',
 'onions',
 'simmer',
 'spinach',
 'wilts',
 'tofu',
 'heated',
 'minutes',
 'season',
 'soup',
 'taste',
 'ladle',
 'soup',
 'bowls',
 'serve']

In [117]:
test_run.iloc[29]

Unnamed: 0                                                            1206
calories                                                               766
categories               ['Sandwich', 'Beef', 'Cheese', 'Garlic', 'Onio...
date                                               2007-07-26 17:48:19.000
desc                     Three make a meal, but a bunch make fun party ...
directions               ['Mix all meats, panko, 1/2 cup water, 6 table...
fat                                                                     43
ingredients              ['1/2 pound ground beef', '1/2 pound ground po...
protein                                                                 36
rating                                                               4.375
sodium                                                                1330
title                                                    Meatball Sliders 
Ingredient_join          [ ' 1 / 2   p o u n d   g r o u n d   b e e f ...
processed_titles         

In [ ]:
model_combo.most_similar(positive=['dried'])

In [ ]:
print (model_combo.similarity(['threads'], 'dried'))

In [ ]:
def recipe_vectorizer(stuff):
    test_list=[]
    for word in stuff:
        test_list.append(model_combo[word])
    return sum(test_list)

In [ ]:
df['recipe_combo'][0]

In [ ]:
recipe_vectorizer(df['recipe_combo'][0])

In [ ]:
sum(test_list)

In [ ]:
total_vocab = set()
for comment in texts:
    total_vocab.update(comment)
len(total_vocab)

In [ ]:
articles_concat = []
for article in texts:
    articles_concat += article

In [ ]:
texts=df['processed_ingredients']

In [ ]:
#x=df[df['Ingredient_join'].str.match('yams')]

In [ ]:
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
index_tmpfile = get_tmpfile("index")
#query = [(1, 2), (6, 1), (7, 2)]

#index = Similarity(index_tmpfile, common_corpus, num_features=len(common_dictionary))  # build the index
#similarities = index[query]  # get similarities between the query and all index documents

In [ ]:
>>> from gensim.test.utils import common_corpus, common_dictionary
>>> from gensim.similarities import MatrixSimilarity
>>>
>>> query = [(1, 2), (5, 4)]
>>> index = MatrixSimilarity(common_corpus, num_features=len(common_dictionary))
>>> sims = index[query]

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
#print(dictionary.token2id)
print(dictionary.values)

In [ ]:
new_doc= vectorize_list[1]
new_vec = dictionary.doc2bow(new_doc)
print(new_vec[3:6])

In [ ]:
trial=new_vec[3:6]

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
#lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

In [ ]:
#vec_lsi = lsi[new_vec] 

In [ ]:
#from gensim import similarities
#index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=12)

In [ ]:
#index = similarities.MatrixSimilarity(lsi[corpus])

In [ ]:
#word2index = {token: token_index for token_index, token in enumerate(w2v.index2word)} 
#word2index['hi'] == 30308  # True

In [ ]:
#model.wv.vocab.get(query_doc[0]).index

In [ ]:
sims = index[vec_lsi]
print(vec_lsi)


In [ ]:
sims = gensim.similarities.Similarity(texts,tf_idf[corpus],
                                      num_features=len(dictionary))
#print(sims)
#print(type(sims))

In [ ]:
query_doc = [w.lower() for w in word_tokenize("lasagna")]
print(query_doc)
query_doc_bow = dictionary.doc2bow(query_doc)
print(query_doc_bow)
query_doc_tf_idf = tf_idf[query_doc_bow]
print(query_doc_tf_idf)

In [ ]:
texts

In [ ]:
#articles_concat
#.to_csv('recipe_processed.csv',index=None)

In [ ]:
import fasttext
from gensim.models.keyedvectors import KeyedVectors

In [ ]:


# Skipgram model
model = fasttext.skipgram('sample_1.txt', 'model')
#print(model.words) # list of words in dictionary

# CBOW model
#model = fasttext.cbow('data.txt', 'model')
#print model.words # list of words in dictionary

In [ ]:
#fasttext skipgram vector list
vector_list=[]
for x in range(len(articles_concat)):
    vector_list.append(model[articles_concat[x]])

In [ ]:
vec_list=[]
for vec in df['vector_ingred_lines']:
    vec_list.append(vec)

In [ ]:
test_ingred=df['processed_ingredients'][8]

In [ ]:
df['processed_ingredients'][8]

In [ ]:
#fasttext skipgram array format
#X=np.array(vector_list)

In [ ]:
X=np.array(vector_5_list)


In [ ]:
from sklearn.cluster import KMeans

k_means = KMeans(n_clusters = 10)
k_means.fit(X)
y_hat = k_means.predict(X)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c = y_hat, s = 10)
cl_centers = k_means.cluster_centers_
plt.scatter(cl_centers[:, 0], cl_centers[:, 1], c='black', s=40);

In [ ]:
from sklearn.metrics.cluster import calinski_harabaz_score, adjusted_rand_score

In [ ]:
calinski_harabaz_score(X, y_hat)

In [ ]:
from nltk.cluster import KMeansClusterer, euclidean_distance

In [ ]:
clusterer = KMeansClusterer(30, euclidean_distance,avoid_empty_clusters=True )
clusters = clusterer.cluster(X, True, trace=True)

In [ ]:
clusters

In [ ]:
#model['fish']

In [ ]:
#for word in model.words:
    #print(word, model[word])

In [ ]:
# CBOW model
#model_cb = fasttext.cbow(texts, 'model')
#print(model_cb.words) # list of words in dictionary

In [ ]:
#word_tokenize(texts)

In [ ]:
model_4=Word2Vec(df,min_count=1)

In [ ]:
model_5=Word2Vec(df['processed_ingredients'],min_count=1)

In [ ]:
model_5['pizza']

In [ ]:
model_5.most_similar(['parmesan'])

In [ ]:
df['processed_ingredients']

In [ ]:
model_4.most_similar(['chicken','parmesan'])

In [ ]:
model_3.most_similar(positive=['chicken','parmesan'])

In [ ]:
model_4.most_similar(positive=['parmesan'])

In [ ]:
print (model_4.similarity('syrup', 'bourbon'))
print (model_4.similarity('garlic', 'shallots'))

In [ ]:


from sklearn import metrics
 
import gensim.models as g
import codecs
 
#model="doc2vec/doc2vec.bin"
test_docs="/Users/denisdunn/usda_project"
 
#inference hyper-parameters
start_alpha=0.01
infer_epoch=1000
 
#load model
#m = g.Doc2Vec.load(model)
#test_docs = [ x.strip().split() for x in codecs.open(test_docs, "r", "utf-8").readlines() ]
m=model
print (test_docs)
"""
[['the', 'cardigan', 'welsh', 'corgi'........
"""
 
X=[]
for d in test_docs:
     
    X.append( m.infer_vector(d, alpha=start_alpha, steps=infer_epoch) )
    
 
k=3
 
from sklearn.cluster import Birch
 
brc = Birch(branching_factor=50, n_clusters=k, threshold=0.1, compute_labels=True)
brc.fit(X)
 
clusters = brc.predict(X)
 
labels = brc.labels_
 
 
print ("Clusters: ")
print (clusters)
 
 
silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')
 
print ("Silhouette_score: ")
print (silhouette_score)

In [ ]:
test_docs

In [ ]:
model = fasttext.load_model('model.bin')
print(model.words) # list of words in dictionary
print(model['bread']) # get the vector of the word 'king'

In [ ]:
model.nn


In [ ]:
classifier = fasttext.supervised('epicurious', 'model')

In [ ]:
classifier = fasttext.supervised('epicurious', 'model', label_prefix='lasagna')

In [ ]:
gensim.models.keyedvectors.BaseKeyedVectors(175)

In [ ]:
print(x)

In [ ]:

model.skipgram(input='epicurious', output='model', epoch=100, lr=0.7)

In [ ]:
fasttext_nn('lasagna')

In [ ]:
result = classifier.test('/Users/denisdunn/usda_project')
print('P@1:', result.precision)
print('R@1:', result.recall)
print('Number of examples:', result.nexamples)


In [ ]:
print(result.bin)

In [ ]:
df["Ingredient_join"][1]

In [ ]:
texts = ["whipping cream onions bay leaves whole cloves garlic clove crushed 1/8 ground nutmeg 'pinch dried thyme crumbled shallots minced butter trimmed boneless center pork loin sinew removed 1-inch chunks well chilled eggs purpose flour tawny port dried currants minced 'lettuce leaves 'cracked peppercorns 'minced parsley 'bay leaves 'french bread baguette slices olive oil red onions halved dried currants red wine vinegar canned chicken broth thyme dried crumbled sugar"]
labels = classifier.predict(texts)
print(labels)

# Or with the probability
labels = classifier.predict_proba(texts)
print(labels)

In [ ]:
sims = index[tfidf[vec_lsi]]
print(list(enumerate(sims)))

In [ ]:
from

In [ ]:
X=model_3

In [ ]:
NUM_CLUSTERS=3
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(X, assign_clusters=True)
print (assigned_clusters)
 
words = list(model.wv.vocab)
for i, word in enumerate(words):  
    print (word + ":" + str(assigned_clusters[i]))
 
 
 
kmeans = cluster.KMeans(n_clusters=NUM_CLUSTERS)
kmeans.fit(X)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
 
print ("Cluster id labels for inputted data")
print (labels)
print ("Centroids data")
print (centroids)
 
print ("Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):")
print (kmeans.score(X))
 
silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')
 
print ("Silhouette_score: ")

In [ ]:

# training data
 
sentences = [df["Ingredient_join"]]
 
 
# training model
model = Word2Vec(sentences, min_count=1)
 
# get vector data
X = model[model.wv.vocab]
#print (X)
 
#print (model.similarity('sugar', 'sauce'))
 
print (model.similarity('chicken'))
 
#print (model.most_similar(positive=['fish'], negative=[], topn=2))
 
#print (model['the'])
 
print (list(model.wv.vocab))
 
print (len(list(model.wv.vocab)))
 
 
 
 
NUM_CLUSTERS=3
kclusterer = KMeansClusterer(direct_vecNUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(X, assign_clusters=True)
print (assigned_clusters)
 
words = list(model.wv.vocab)
for i, word in enumerate(words):  
    print (word + ":" + str(assigned_clusters[i]))
 
 
 
kmeans = cluster.KMeans(n_clusters=NUM_CLUSTERS)
kmeans.fit(X)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
 
print ("Cluster id labels for inputted data")
print (labels)
print ("Centroids data")
print (centroids)
 
print ("Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):")
print (kmeans.score(X))
 
silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')
 
print ("Silhouette_score: ")

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.similarities import WmdSimilarity

model = Word2Vec(texts, size=20, min_count=1)  # train word-vectors

index = WmdSimilarity(texts, model)
 # Make query.
query = ['broccoli']
sims = index[query]

In [ ]:
word_vectors = model.wv.vectors

In [ ]:
sims.max()

In [ ]:

from sklearn.cluster import KMeans

In [ ]:
idx =k_means.fit_predict( word_vectors)

In [ ]:
idx.shape

In [ ]:
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [ ]:
words = list(model.vocab)
for i, word in enumerate(words):  
    print (word + ":" + str(assigned_clusters[i]))

In [ ]:
#for cluster in xrange(0,10):

# Print the cluster number

#…………….print "\nCluster %d" % cluster

# Find all of the words for that cluster number, and print them out

words = []

for i in range(0,len(word_centroid_map.values())):

    if( word_centroid_map.values()[i] == cluster ):

        words.append(word_centroid_map.keys()[i])
print(words)

In [ ]:
plt.scatter(idx[:,0],idx[:1], c=8, s=10)
centers = k_means.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=70)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [ ]:
nb_classifier.fit(tf_idf_data_train, target)
nb_train_preds = nb_classifier.predict(tf_idf_data_train)
nb_test_preds = nb_classifier.predict(tf_idf_data_test)

In [ ]:
nb_classifier = MultinomialNB()
rf_classifier = RandomForestClassifier(n_estimators=100)

In [ ]:
from sklearn.decomposition.truncated_svd import TruncatedSVD        
pca = TruncatedSVD(n_components=2)                                
X_reduced_train = pca.fit_transform(index) 